In [ ]:
import cv2 
import numpy as np 
import os
from matplotlib import pyplot as plt 
import time 
import mediapipe as mp 
import shutil


In [ ]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [ ]:
def mediapipe_detection(image, model): 
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Color Conversion BGR 2 RGB 
    image.flags.writeable = False  #Image is no longer writeable 
    results = model.process(image) # Make predicition
    image.flags.writeable = True # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # Color Conversion RGB 2 BGR 
    return image,results

In [ ]:
def draw_landmarks(image, results): 
    # Draw Face Connections
    #The 4th parameter is going to draw the landmarks and the 5th the connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS, 
                              mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius = 1 ), 
                              mp_drawing.DrawingSpec(color=(80,256,51), thickness=1, circle_radius=1)) 
    # Draw Pose Connections 
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius = 4 ), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2))
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                              mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius = 4 ), 
                              mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)) 
    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                              mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius = 4 ), 
                              mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                              ) 

In [ ]:
# Extracting the keypoints 
def extract_keypoints(results): 
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)

    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
 
    return np.concatenate([pose,face,lh,rh])

In [ ]:
dir_path = "../DataSet" 
actions = np.array(os.listdir(dir_path))


results_path = os.path.join('data_results')



for action in actions: 
    num_of_videos = os.listdir(dir_path+'/'+action)
    for video in range(len(num_of_videos)): 
        try: 
            os.makedirs(os.path.join(results_path, action, str(video)))
        except: 
            pass 


In [ ]:
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) 
test_actions = ['Blue','Happy','Man', 'Family']
for action in test_actions: 
    file_list = os.listdir(dir_path+'/'+action)
    num_files = len(file_list)

    for file in range(num_files):
        file_name = file_list[file]
        cap = cv2.VideoCapture(dir_path+'/'+action+'/'+file_name)
        num_of_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        for i in range(num_of_frames): 
            ret, frame = cap.read()

            if not ret:
                break 

            image, results = mediapipe_detection(frame, holistic)
            cv2.imshow('',image)
            print(results)

            draw_landmarks(image,results)

            keypoints = extract_keypoints(results)
            npy_path = os.path.join(results_path, action, str(file), str(i) )
            np.save(npy_path, keypoints)

cap.release()
cv2.destroyAllWindows()
